In [62]:
class Item:
    def __init__(self, item: str, utility: int):
        self.item = item
        self.utility = utility

    def __repr__(self):
        return f"{self.item}"

class Transaction:
    def __init__(self, id: int, items_quantities: dict):
        if any(q <= 0 for q in items_quantities.values()):
            raise ValueError(f"Quantities in trans{id} must be positive integers.")
        self.id = id
        self.items_quantities = items_quantities

    def __repr__(self):
        return (f"(tid = {self.id}, frequencies = {self.items_quantities})")


class Utilities:
    def __init__(self, tid: int, iutil: int, rutil: int):
        self.tid = tid
        self.iutil = iutil
        self.rutil = rutil

    def __repr__(self):
        return f"Utilities(tid={self.tid}, iutil={self.iutil}, rutil={self.rutil})"


class UtilityList:
    def __init__(self, items: set[Item], utility_list: list[Utilities]):
        self.items = items
        self.utility_list = utility_list

    def __repr__(self):
        return f"UtilityList(items={self.items}, utility_list={self.utility_list})"

In [63]:
def calculate_positive_utility_of_transaction(trans: Transaction):
    pu = 0
    items: set[Item] = trans.items_quantities.keys()
    for item in items:
        if item.utility > 0:
            pu += trans.items_quantities.get(item) * item.utility
    return pu

In [64]:
def calculate_transaction_weight_utility(items: set[Item], database: list[Transaction]):
    twu = 0
    for trans in database:
        trans_items = trans.items_quantities.keys()
        if(items.issubset(trans_items)):
            twu += calculate_positive_utility_of_transaction(trans)
    return twu

In [65]:
a = Item("A", 5)
b = Item("B", 2)
c = Item("C", 1)
d = Item("D", 2)
e = Item("E", 3)

trans1 = Transaction(1, dict(zip([a, c], [1, 1])))
trans2 = Transaction(2, dict(zip([e], [1])))
trans3 = Transaction(3, dict(zip([a, b, c, d, e], [1, 5, 1, 3, 1])))
trans4 = Transaction(4, dict(zip([b, c, d, e], [4, 3, 3, 1])))
trans5 = Transaction(5, dict(zip([a, c, d], [1, 1, 1])))
trans6 = Transaction(6, dict(zip([a, c, e], [2, 6, 2])))
trans7 = Transaction(7, dict(zip([b, c, e], [2, 2, 1])))

items = [a, b, c, d, e]
database = [trans1, trans2, trans3, trans4, trans5, trans6, trans7]

In [66]:
def find_all_transactions_contained_item_set(
    item_set: set[Item], database: list[Transaction]
):
    transactionsList: list[Transaction] = []
    for trans in database:
        if item_set.issubset(trans.items_quantities.keys()):
            transactionsList.append(trans)
    return transactionsList

def find_max_min_avg_periodic_of_item_set(
    item_set: set[Item], database: list[Transaction]
):
    contained_transactions = find_all_transactions_contained_item_set(
        item_set, database
    )
    trans_ids: list[int] = [trans.id for trans in contained_transactions]
    m = len(contained_transactions)
    ps: list[int] = []
    for i in range(0, m + 1):
        if i == 0:
            ps.append(trans_ids[0])
        elif i == m:
            ps.append(len(database) - trans_ids[m - 1])
        else:
            ps.append(trans_ids[i] - trans_ids[i - 1])
  
    print(ps)
    max_per, min_per = ps[0], ps[0]
    avg_per = len(database) / len(ps)
    for i in ps:
        if i > max_per:
            max_per = i
        if i < min_per:
            min_per = i
    return max_per, min_per, avg_per

In [67]:
print(calculate_transaction_weight_utility({a, b}, database))

25


In [68]:
def check_order_item_and_set(ik: Item, X: set[Item]) -> bool:
    twu = calculate_transaction_weight_utility()
    for item in X:
        if ik.item <= item.item:  
            return False
    return True

def calculate_remaining_utility_of_item_set_in_transaction(
    items: set[Item], trans: Transaction
):
    ru = 0
    trans_items: set[Item] = set(trans.items_quantities.keys())
    for item in trans_items:
        if check_order_item_and_set(item, items) == True:
            ru += item.utility * trans.items_quantities.get(item)
    return ru

In [69]:
def calculate_utility_of_item_set_in_trans(
    items: set[Item], trans: Transaction
):
    pu = 0
    for item in items:
        quantity = trans.items_quantities.get(item)
        if quantity != None:
            pu += item.utility * quantity
    return pu

In [70]:
def create_utility_utility_list(item_set: set[Item], database: list[Transaction]): 
    utils_list: list[Utilities] = []
    for trans in database:
        if(item_set.issubset(trans.items_quantities.keys())):
            iutil = calculate_utility_of_item_set_in_trans(item_set, trans)
            rutil = calculate_remaining_utility_of_item_set_in_transaction(item_set, trans)
            utils = Utilities(trans.id, iutil, rutil)
            utils_list.append(utils)

    return UtilityList(item_set, utils_list)

In [71]:
test = {b, e}
create_utility_utility_list(test, database)

UtilityList(items={B, E}, utility_list=[Utilities(tid=3, iutil=13, rutil=0), Utilities(tid=4, iutil=11, rutil=0), Utilities(tid=7, iutil=7, rutil=0)])

In [72]:
def find_max_min_avg_periodic_of_item_set(
    item_set: set[Item], database: list[Transaction]
):
    contained_transactions = find_all_transactions_contained_item_set(
        item_set, database
    )
    trans_ids: list[int] = [trans.id for trans in contained_transactions]
    m = len(contained_transactions)
    ps: list[int] = []
    for i in range(0, m + 1):
        if i == 0:
            ps.append(trans_ids[0])
        elif i == m:
            ps.append(len(database) - trans_ids[m - 1])
        else:
            ps.append(trans_ids[i] - trans_ids[i - 1])
    max_per, min_per = ps[0], ps[0]
    avg_per = len(database) / len(ps)
    for i in ps:
        if i > max_per:
            max_per = i
        if i < min_per:
            min_per = i
    return max_per, min_per, avg_per